Task 1: Third-Order Letter Approximation Model

In [1]:
# Imports.

# Selecting random items from lists.
import random

# Efficient data structures.
import collections

Step 1: A method that will read in the text from one of the books

In [2]:
#method to read in a book
def readBook(book):
    with open('books/'+book, 'r', encoding='utf-8') as file:#open the file
        english = file.read()#store the read in file in a variable

    return english#return the text

In [3]:
#Clean the text by putting it to lower case and only keeping ascii chars
def cleanBook(book):
    #change everything to lower case
    english = book.upper();
    # The characters to keep.
    keep = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ .'
    # Remove unwanted characters.
    cleaned = ''.join(c for c in english if c in keep)

    return cleaned


In [5]:
text = readBook('voyaging.txt')
cleanedText = cleanBook(text)
print(cleanedText)

                                     .                        ...                         .                                     . .            ..                 .                                                                                                                                                                                                                                     . .                                                                                                              . .                                                                                                                                                                                                                                                                                 .            .                                      .                           .                      .                      .                                .                 .          .                              